In [ ]:
# pip install transformers torch scikit-learn numpy sentence-transformers ipywidgets

Note: you may need to restart the kernel to use updated packages.


d:\github\year4\nlp\project\boardgame-scraping\.venv\Scripts\python.exe: No module named pip


In [17]:
from transformers import AutoTokenizer, AutoModel
import torch
import os
import numpy as np
from sentence_transformers import SentenceTransformer
import csv
import json
from pymilvus import connections, Collection, FieldSchema, CollectionSchema, utility, DataType
import os

In [18]:
model_name = "Alibaba-NLP/gte-multilingual-base"

In [19]:
bge_model = SentenceTransformer(model_name, trust_remote_code=True)


Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
def generate_embedding(text):
    # Use the SentenceTransformer model to generate embeddings
    embeddings = bge_model.encode(text,convert_to_numpy=False)
    return embeddings

In [ ]:

MILVUS_HOST = 'localhost'
MILVUS_PORT = '19530'

connections.connect("default", host=MILVUS_HOST, port=MILVUS_PORT)

In [3]:
model_dim = 768

In [9]:
# Function to initialize Milvus collection
def initialize_milvus_collection(document_name):
    # Check if collection exists
    if not utility.has_collection(document_name):
        # Create collection if it doesn't exist
        # You may need to adjust the schema based on your specific requirements

        fields = [
            FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
            # FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=1024),  # Adjust dim if needed
            FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=model_dim),  # Adjust dim if needed

            FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=65535)
        ]
        schema = CollectionSchema(fields, "Document embeddings for Information database")
        collection = Collection(document_name, schema)
        
        # Create an IVF_FLAT index for the embedding field
        index_params = {
            "metric_type": "L2",
            "index_type": "IVF_FLAT",
            "params": {"nlist": model_dim}
        }
        collection.create_index("embedding", index_params)
    else:
        collection = Collection(document_name)
    
    return collection

In [10]:
# Initialize Milvus collection
collection_name = 'note_book'
collection = initialize_milvus_collection(collection_name)
collection.load()

In [22]:
def insert_data_to_milvus(text):
    # Generate embeddings for the text
    embedding = generate_embedding(text)

    # Prepare the entity to be inserted
    entity = {
        "text": text,
        "embedding": embedding
    }


    # Insert the entity into Milvus
    insert_result = collection.insert([entity])


    # Ensure the changes are immediately searchable
    collection.flush()

In [23]:
def read_and_index_docs(docs_dir='../docs'):
    """
    Read .txt files from the specified directory and index their contents.
    
    :param docs_dir: Directory containing the .txt files
    :param index_url: URL of the indexing endpoint
    """
    indexed_files = 0
    non_indexed_files = 0
    
    selected_lines = ['Meta data', 'Content']
    for filename in os.listdir(docs_dir):
        if filename.endswith('.txt'):
            file_path = os.path.join(docs_dir, filename)
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                    
                # Split content into chunks of max 1000 characters, including the title in each chunk
                chunks = []
                lines = content.split('\n')
                title = lines[0]  # Get the first line as the title
                current_chunk = title + '\n'  # Start each chunk with the title
                for line in lines[1:]:  # Skip the first line (title) in this loop
                    if any(line.startswith(prefix) for prefix in selected_lines):
                        if len(current_chunk) + len(line) + 1 <= 1000:  # +1 for newline
                            current_chunk += line + '\n'
                        else:
                            if current_chunk:
                                chunks.append(current_chunk.strip())
                            current_chunk = title + '\n' + line + '\n'  # Start a new chunk with the title
                if current_chunk:
                    chunks.append(current_chunk.strip())

                # Index each chunk separately
                file_indexed = True
                for chunk in chunks:
                    insert_data_to_milvus(chunk)
                
                if file_indexed:
                    indexed_files += 1
                    print(f"Finished indexing all chunks from {filename}")
                else:
                    non_indexed_files += 1
            
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")
                non_indexed_files += 1

    print(f"Indexing complete. Indexed files: {indexed_files}, Non-indexed files: {non_indexed_files}")

In [24]:
read_and_index_docs()

Finished indexing all chunks from 7 Wonders (TH) 7 สิ่งมหัศจรรย์.txt
Finished indexing all chunks from 7 Wonders Architects (TH) 7 สิ่งมหัศจรรย์ ยอดสถาปนิก.txt
Finished indexing all chunks from 8 BIT BOX (TH) 8 บิทบ๊อกซ์.txt
Finished indexing all chunks from Art Society (TH) ศิลป์สโมสร.txt
Finished indexing all chunks from Bandido (THEN) แบนดิโด.txt
Finished indexing all chunks from Betakkuma  Fart & Furious ตดทะลุนรก.txt
Finished indexing all chunks from Bloodbound (TH) สงครามแวมไพร์.txt
Finished indexing all chunks from Bloody Inn (TH) โรงแรมสีเลือด.txt
Finished indexing all chunks from Bloody Inn Carnies (TH) โรงแรมสีเลือด ชาวคณะหรรษา.txt
Finished indexing all chunks from Book - Captive (TH).txt
Finished indexing all chunks from Book - Knight (TH).txt
Finished indexing all chunks from Book - Sherlock Holmes & Moriarty Associates (TH) เชอร์ล็อคโฮล์มส์ & มอริอาร์ตี้ พันธมิตรอันตราย.txt
Finished indexing all chunks from Book - Sherlock Holmes four investigations (TH).txt
Finished index

In [ ]:
def completions(query):     
    query_embedding = generate_embedding(query).numpy().flatten().tolist()
    # Prepare search parameters
    search_param = {
        "metric_type": "L2",
        "params": {"nprobe": 10},
    }
    # Step 2: Retrieve top-10 documents from Milvus
    search_results = collection.search(
        data=[query_embedding],
        anns_field="embedding",
        param=search_param,
        limit=10,
        output_fields=["id", "text", "embedding"],
        expr=None
    )
    
    # Extract document texts and embeddings
    retrieved_documents = []
    document_embeddings = []
    for hits in search_results:
        for hit in hits:
            logger.debug(f"Retrieved document: {hit.entity.get('text')[:50]}...")
            retrieved_documents.append(hit.entity)
            embedding = hit.entity.get('embedding')
            logger.debug(f"Retrieved embedding: {embedding[:5]}... (truncated)")
            if embedding is not None:
                document_embeddings.append(embedding)
                
    return retrieved_documents, document_embeddings